# Real Estate Fair Value

## Morgage

Owner:

$rent - depositO*adjDepositInterest >= (propertyCost - depositO)*morgageInterest + expensesO - adjGain$

Investor:

$(rent - expensesI - (propertyCost - depositI)*morgageInterest)*(1 - tax) + adjGain >= depositI*adjDepositInterest$

Assuming the property inflation is equal to inflation on a long term.

In [210]:
propertyCost = 500000 #500k
rent = 450*52 # $500 weekly
rates = 3500

morgageInterest = 0.052 #5.2%
depositInterest = 0.025 #2.5%

inflation = 0.017 #1.7%
propertyInflation = inflation #0.02

depositPercentO = 0.2 # 20% Current LVR NZ
depositPercentI = 0.3 # 30% Current LVR NZ

In [207]:
tax = 0.28 #High PIR NZ 28%

ratesRate = rates/propertyCost
insuranceRateO = 0.002 #0.2%
insuranceRateI = 0.003 #0.3%
maintainance = 1100
managementRentRate = 0.08 #8% of annual rent

adjDepositInterest = depositInterest*(1 - tax) - inflation
print("adjDepositInterest = " + str(round(adjDepositInterest * 100, 2)) + "%")

adjGainRate = propertyInflation - inflation
adjGain = propertyCost*adjGainRate

rates = propertyCost*ratesRate
insuranceO = propertyCost*insuranceRateO
insuranceI = propertyCost*insuranceRateI
management = rent*managementRentRate

expensesO = rates + insuranceO + maintainance
expensesI = rates + insuranceI + management + maintainance
print("rent = $" + str(rent))
print("expensesO = $" + str(expensesO), "expensesI = $" + str(expensesI))

adjDepositInterest = 0.1%
rent = $23400
expensesO = $5600.0 expensesI = $7972.0


In [211]:
def checkFormulas(depositPercentO=depositPercentO, depositPercentI = depositPercentI):
    depositO = propertyCost*depositPercentO
    depositI = propertyCost*depositPercentI
    leftO = rent - depositO*adjDepositInterest 
    rightO = (propertyCost - depositO)*morgageInterest + expensesO - adjGain
    print("Owner =", leftO >= rightO, "=", round(leftO), ">=", round(rightO))
    
    leftI = (rent - expensesI - (propertyCost - depositI)*morgageInterest)*(1 - tax) + adjGain
    rightI = depositI*adjDepositInterest
    print("Investor =", leftI >= rightI, "=", round(leftI), ">=", round(rightI))
checkFormulas()

Owner = False = 23300 >= 26400
Investor = False = -1996 >= 150


## No morgage 

$deposit == propertyCost$

Owner: $rent + adjGain - propertyCost*adjDepositInterest >= expenses$

Or: $rent + adjGain - expenses >= propertyCost*adjDepositInterest$

Investor: $(rent - expenses)*(1 - tax) + adjGain >= propertyCost*adjDepositInterest$

In [175]:
checkFormulas(depositPercentO=1, depositPercentI=1)

Owner = True = 22984 >= 5432
Investor = True = 11290 >= 416


## Removing property cost

We can remove the property cost from the equasion with:

Owner:

$rentInterest - depositPercentO*adjDepositInterest >= (1 - depositPercentO)*morgageInterest + expenseRateO - adjGainRate$

$rentInterest + adjGainRate - adjDepositInterest >= expenseRateO$

Investor:

$(rentInterest - expenseRateI - (1 - depositPercentI)*morgageInterest)*(1 - tax) + adjGainRate >= depositPercent*adjDepositInterest$

$(rentInterest - expenseRateI)*(1 - tax) + adjGainRate >= adjDepositInterest$

In [158]:
rentInterest = rent/propertyCost
expensesRateO = ratesRate + insuranceRateO + maintainance/propertyCost
expensesRateI = ratesRate + insuranceRateI + managementRentRate*rent/propertyCost + maintainance/propertyCost
print("rentInterest =", str(round(rentInterest*100, 2)) + "%", 
      "adjDepositInterest =", str(round(adjDepositInterest*100, 2)) + "%",
      "expensesRateO =", str(round(expensesRateO*100, 2)) + "%",
      "expensesRateI =", str(round(expensesRateI*100, 2)) + "%",
      "adjGainRate =", str(round(adjGainRate*100, 2)) + "%",
     )



rentInterest = 4.68% adjDepositInterest = 0.1% expensesRateO = 1.12% expensesRateI = 1.59% adjGainRate = 0.0%


## Intrinsic value

In [243]:
def intrinsicCost(rent, rates, 
                  depositPercent = 0.2, morgageInterest = 0.052, depositInterest = 0.025, 
                  insuranceRate = 0.002, maintanance = 1100, 
                  inflation = 0.017): 
    return (rent - rates - maintanance)/(morgageInterest - depositPercent*morgageInterest + insuranceRate + depositPercent*(depositInterest*(1 - tax) - inflation))
#rent - propertyCost*depositPercentO*adjDepositInterest >= (propertyCost - propertyCost*depositPercentO)*morgageInterest + rates + insuranceRateO*propertyCost + maintainance - adjGain
#intrinsicCost = (rent + adjGain - expensesO)/ (morgageInterest - depositPercentO*morgageInterest + depositPercentO*adjDepositInterest)
intrinsicCost = intrinsicCost(rent, rates)
print("intrinsicCost =", str(round(intrinsicCost/1000)) + "k")

intrinsicCost = 429k


## Deposit percent calculation

In [146]:
startDepositPercentO = (expensesRateO + morgageInterest - rentInterest - adjGainRate)/ (morgageInterest - adjDepositInterest)
startDepositPercentI = ((expensesRateI + morgageInterest - rentInterest)*(1 - tax) - adjGainRate)/(morgageInterest*(1 - tax) - adjDepositInterest)
print("depositPercentO >= ", str(round(startDepositPercentO*100)) + "%,", 
      "depositPercentI >= ", str(round(startDepositPercentI*100)) + "%")

checkFormulas()

depositPercentO >=  32%, depositPercentI >=  42%
Owner = False = 23300 >= 26400
Investor = False = -1996 >= 150


In [147]:
import pandas as pd

## Inflation
For older data infration will need to be applied. CPI Level 2 Subgroups for New Zealand (Qrtly-Mar/Jun/Sep/Dec)
http://archive.stats.govt.nz/infoshare/SelectVariables.aspx?pxID=4cd6f132-2491-456d-aa7d-1900801982ad

In [148]:
inflation = pd.read_csv('inflation.csv', index_col=0, header=1, skipfooter=33, engine='python', parse_dates=True, na_values='..')
inflation = inflation[['Home ownership', 'Property maintenance', 'Property rates and related services', 'Household energy',
       'Furniture, furnishings and floor coverings', 'Household textiles',
       'Household appliances', 'Glassware, tableware and household utensils',
       'Tools and equipment for house and garden',
       'Other household supplies and services']].dropna()
inflation['Overall Property'] = inflation.sum(axis=1)
print("Household categories", inflation.columns.values)
inflation.tail()

Household categories ['Home ownership' 'Property maintenance'
 'Property rates and related services' 'Household energy'
 'Furniture, furnishings and floor coverings' 'Household textiles'
 'Household appliances' 'Glassware, tableware and household utensils'
 'Tools and equipment for house and garden'
 'Other household supplies and services' 'Overall Property']


,Home ownership,Property maintenance,Property rates and related services,Household energy,"Furniture, furnishings and floor coverings",Household textiles,Household appliances,"Glassware, tableware and household utensils",Tools and equipment for house and garden,Other household supplies and services,Overall Property
2018-01-01,1028.0,1035.0,1031.0,1012.0,1025.0,987.0,975.0,950.0,1001.0,1010.0,10054.0
2018-04-01,1039.0,1041.0,1031.0,1028.0,1015.0,969.0,982.0,969.0,990.0,1018.0,10082.0
2018-07-01,1052.0,1048.0,1077.0,1027.0,1022.0,998.0,983.0,976.0,979.0,1035.0,10197.0
2018-10-01,1061.0,1055.0,1078.0,1027.0,1050.0,971.0,966.0,910.0,970.0,1029.0,10117.0
2019-01-01,1068.0,1058.0,1078.0,1039.0,992.0,932.0,992.0,954.0,989.0,1022.0,10124.0


## Maintenance cost
Source: https://www.scoop.co.nz/stories/BU1703/S00953/property-maintenance-costs-surprising.htm

In [149]:
maitenanceDf = pd.read_csv('maintenance.csv', index_col=0)
maitenanceDf.head()

,annual_cost
property_type,
Appartment,728
Unit,784
Flat,805
Townhouse,921
House,1024
